#Setting up environment and preprocessing Dataset

##Installing needed packages and importing Libraries

In [ ]:
# GitHub with the Python codes for Keras segmentation models:
! pip install git+https://github.com/Macnolio/Satellite-Part-Images-Segmentation

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/Macnolio/Satellite-Part-Images-Segmentation to /tmp/pip-req-build-1g7mik1d
  Running command git clone -q https://github.com/Macnolio/Satellite-Part-Images-Segmentation /tmp/pip-req-build-1g7mik1d
     |████████████████████████████████| 2.9 MB 22.9 MB/s 
     |████████████████████████████████| 3.3 MB 30.6 MB/s 
  Created wheel for keras-segmentation: filename=keras_segmentation-0.3.0-py3-none-any.whl size=34604 sha256=5e872f80aad8a2e1dd1c9e62fde74ae87b662a2710e6e703eca7f54d1f69c1dc
  Stored in directory: /tmp/pip-ephem-wheel-cache-5osr0oe3/wheels/9a/fb/1b/02d07fcd8b1265c2f71d8c6855e217cc87bce58ece9208e58c
Successfully built keras-segmentation
  Attempting uninstall: imageio
    Found existing installation: imageio 2.9.0
    Uninstalling imageio-2.9.0:
      Successfully uninstalled imageio-2.9.0
  Attempting uninstall: h5py
    Found existing installation: h5p

##Downloading Dataset

In [ ]:
#Mounting Personal Drive into Google Collab: Before upload the Final_Dataset.rar to your personal drive.
from google.colab import drive
drive.mount('/content/gdrive')

#Asking for download of the dataset:
import requests
file_url = "https://drive.google.com/file/d/1x0zBjpGVBorbFLjpcIlK8Wr1E68yTYL7/view?usp=sharing" #Share link of the dataset, change it to your own link

r = requests.get(file_url, stream = True)

with open("/content/gdrive/My Drive/python.pdf", "wb") as file:
    for block in r.iter_content(chunk_size = 1024):
         if block:
             file.write(block)

# Other option if you are the main user of the drive: It starts failing when too it is used many times
#!gdown 1x0zBjpGVBorbFLjpcIlK8Wr1E68yTYL7

Mounted at /content/gdrive


In [ ]:
#Copying file from the personal drive to collab folder, for easier access:
!cp "/content/gdrive/MyDrive/Final_dataset.rar" "/content/Final_dataset.rar"

In [ ]:
# Unzipping the dataset
!unrar x Final_dataset.rar

## Preprocessing the images

### Training images

In [ ]:
# Importing image and arrays manipulation libraries:
import cv2 as cv
import numpy as np
import os
os.getcwd()

# Renaming masks: Images and masks must have the same name to allow the training function to detect the solution masks
collection = "Final_dataset/train/masks/"
new_name = "Final_dataset/train/images/"
i = 0
for i, filename in enumerate(os.listdir(collection)):
    old_name = filename
    fixed_name = filename.replace("_mask","")
    os.rename("Final_dataset/train/masks/" + old_name , "Final_dataset/train/masks/" + fixed_name)

#Normalizing and resizing training dataset:
# Masks:
mask_path = "Final_dataset/train/masks/"
i = 0
for i, filename in enumerate(os.listdir(mask_path)):
    name = filename
    img = cv.imread(mask_path + name)         # Original masks go between 0 y 11
    # Resizing
    img = cv.resize(img, (576,320), interpolation = cv.INTER_NEAREST) # The images size must be divisible by 32, no other restriction. Reducing the size for reducing training computation cost
    #Reasigning pixel values to each class: The Keras functions work with a unique number in all the RGB channels
    img[np.all(img == (255, 0, 0), axis=-1)] = (1,1,1) #Antenna
    img[np.all(img == (0, 255, 0), axis=-1)] = (2,2,2) #Body
    img[np.all(img == (0, 0, 255), axis=-1)] = (3,3,3) #Solar panel
    # Saving
    cv.imwrite(mask_path + name, img)

# Images:
images_path = "Final_dataset/train/images/"
i = 0
for i, filename in enumerate(os.listdir(images_path)):
    name = filename
    img = cv.imread(images_path + name)
    img = cv.resize(img, (576,320))                        #No need for resizing, the images must be RGB from [0:255] values
    cv.imwrite(images_path + name, img)

###Validation images

In [ ]:
# Importing image and arrays manipulation libraries:
import cv2 as cv
import numpy as np
import os
os.getcwd()

# Renaming masks: Images and masks must have the same name to allow the training function to detect the solution masks
collection = "Final_dataset/val/masks/"
new_name = "Final_dataset/val/images/"
i = 0
for i, filename in enumerate(os.listdir(collection)):
    old_name = filename
    fixed_name = filename.replace("_mask","")
    os.rename("Final_dataset/val/masks/" + old_name , "Final_dataset/val/masks/" + fixed_name)

#Normalizing and resizing training dataset:
# Masks:
mask_path = "Final_dataset/val/masks/"
i = 0
for i, filename in enumerate(os.listdir(mask_path)):
    name = filename
    img = cv.imread(mask_path + name)        #Original masks go between 0 y 11
    # Resizing
    img = cv.resize(img, (576,320), interpolation = cv.INTER_NEAREST) # The images size must be divisible by 32, no other restriction. Reducing the size for reducing training computation cost
    #Reasigning pixel values to each class: The Keras functions work with a unique number in all the RGB channels
    img[np.all(img == (255, 0, 0), axis=-1)] = (1,1,1) #Antenna
    img[np.all(img == (0, 255, 0), axis=-1)] = (2,2,2) #Body
    img[np.all(img == (0, 0, 255), axis=-1)] = (3,3,3) #Solar panel    Tenia axis -1
    # Saving
    cv.imwrite(mask_path + name, img)

# Images:
images_path = "Final_dataset/val/images/"
i = 0
for i, filename in enumerate(os.listdir(images_path)):
    name = filename
    img = cv.imread(images_path + name)
    img = cv.resize(img, (576,320))                           #No need for resizing, the images must be RGB from [0:255] values
    cv.imwrite(images_path + name, img)

### Saving preprocessed dataset

In [ ]:
# Saving the modified data: (I have it in my computer if needed can be imported directly, to skip steps)
## Zipping data
# !zip -r /content/Final_dataset.zip /content/Final_dataset
## Downloading from google colab:
# from google.colab import files
# files.download("Final_dataset.zip")

In [ ]:
#Copying file from the personal drive to collab folder, for easier access:
# !cp "/content/Final_dataset.zip" "/content/gdrive/MyDrive/Final_dataset.zip" 

# Model Definition

In [ ]:
# Importing model from keras_segmentation:
from keras_segmentation.models.fcn import fcn_8_mobilenet
# Initialising the model:
model = fcn_8_mobilenet(n_classes=4 ,  input_height=320, input_width=576)   # Remember: Same size as the resizing

# Check all models in all_models.py: The input size values msut be divisible between 32!!
##from keras_segmentation.models.pspnet import pspnet (Resize to 192 multiple, it only works like that)
##from keras_segmentation.models.pspnet import unet
##from keras_segmentation.models.fcn import fcn_8_mobilenet

In [ ]:
#Summary of the model:
model.summary()     #Prints a table with the model layers 

#Training Model

##Setting up enviroment

###Monitoring and Stopping options:

In [ ]:
# Monitoring and callbacks activation: If the validation_loss keeps increasing, the training is stopped. This aloows to avoid overweight.
from keras import callbacks
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",
										mode ="min", patience = 5,
										restore_best_weights = True)

### Data Augmentation options:

In [ ]:
#Importing data augmentation library:
from imgaug import augmenters as iaa

# Available augmentations:
# augmentation_functions = {
#     "aug_all": _load_augmentation_aug_all,
#     "aug_all2": _load_augmentation_aug_all2,
#     "aug_geometric": _load_augmentation_aug_geometric,
#     "aug_non_geometric": _load_augmentation_aug_non_geometric
# }


## Training Model

In [ ]:
# Using the Github:

# Importing optimizers: 
from keras.optimizers import Adam, SGD

# Training Model
history = model.train(
              train_images =  "Final_dataset/train/images/",
              train_annotations = "Final_dataset/train/masks/",
              n_classes = 4,
              checkpoints_path = "/tmp/fcn_8_mobilenet",   
              epochs=10,
              batch_size=32,
              validate=True,
              val_images="Final_dataset/val/images/",
              val_annotations="Final_dataset/val/masks/",
              val_batch_size=32,
              steps_per_epoch=100,
              val_steps_per_epoch=100,
              optimizer_name=Adam(learning_rate=1e-3),
              do_augment=False,
              augmentation_name="aug_all",
              callbacks=[earlystopping],
              read_image_type=1)

Verifying training dataset


100%|██████████| 2517/2517 [00:19<00:00, 130.46it/s]


Dataset verified! 
Verifying validation dataset


100%|██████████| 600/600 [00:04<00:00, 126.49it/s]


Dataset verified! 
Epoch 1/10
100/100 [==============================] - 434s 4s/step - loss: 0.6706 - accuracy: 0.7457 - val_loss: 0.4319 - val_accuracy: 0.8417
Epoch 2/10
100/100 [==============================] - 415s 4s/step - loss: 0.2108 - accuracy: 0.9121 - val_loss: 0.2398 - val_accuracy: 0.9097
Epoch 3/10
100/100 [==============================] - 414s 4s/step - loss: 0.1511 - accuracy: 0.9458 - val_loss: 0.2144 - val_accuracy: 0.9262
Epoch 4/10
 60/100 [=================>............] - ETA: 2:07 - loss: 0.1144 - accuracy: 0.9601

# Predicting Segmentations

## Only one Image prediction:

In [ ]:
# Importing prediction colors:
from keras_segmentation.data_utils.data_loader import class_colors

# Changing class colors to make them as the original masks
class_colors[0] = (0,0,0)      # Background                               # Current colors selected randomly with a constant seed:  
class_colors[1] = (255,0,0)    # Antenna                                  #(197, 215, 20), (132, 248, 207), (155, 244, 183), (111, 71, 144)
class_colors[2] = (0,255,0)    # Body
class_colors[3] = (0,0,255)    # Solar Panel                                        

# Predicting new segmentation:
o = model.predict_segmentation(
    inp="Final_dataset/val/images/img_resize_677.png",                    # Choose image to be predicted
    out_fname="/tmp/out.png" , overlay_img=False, show_legends=True,      # overlay: Puts the original image behind
    class_names = [ "Background", "Antenna",  "Body", "Solar Panels"])    # legends: Color and corresponding class
                                                                            #438, 637,677
#Printing image:
from IPython.display import Image
Image('/tmp/out.png')


## Predicting a complete dataset

In [ ]:
from keras_segmentation.data_utils.data_loader import class_colors
# Changing class colors to make them as the original masks: This is to be able to evaluate the model afterwards
class_colors[0] = (0,0,0)    # Background
class_colors[1] = (1,1,1)    # Antenna
class_colors[2] = (2,2,2)    # Body
class_colors[3] = (3,3,3)    # Solar Panel

In [ ]:
from keras_segmentation.predict import predict_multiple, predict
predict_multiple(inp_dir="Final_dataset/val/images/", checkpoints_path="/tmp/fcn_32_mobilenet",out_dir="/tmp/prediction")

# Evaluating the model predictions:

## Fast Evaluation

In [ ]:
from keras_segmentation.data_utils.data_loader import class_colors
# Changing class colors to make them as the original masks: This is to be able to evaluate the model afterwards
class_colors[0] = (0,0,0)    # Background
class_colors[1] = (1,1,1)    # Antenna
class_colors[2] = (2,2,2)    # Body
class_colors[3] = (3,3,3)    # Solar Panel

In [ ]:
val_image_dir = "Final_dataset/val/images/"
val_masks_dir = "Final_dataset/val/masks/"
from keras_segmentation.predict import evaluate
ev = model.evaluate_segmentation(inp_images_dir=val_image_dir, annotations_dir=val_masks_dir)
assert ev['frequency_weighted_IU'] > 0.01
print(ev)